## 파이토치

- Inception V2의 각 컨볼루션 레이어 다음에 BN을 넣고 Auxiliary classifer를 붙인 게 곧 Inception V3. 따라서 V2를 만들고 싶으면 두 기능을 주석 처리하면 된다.


- 각 컨볼루션 다음에는 ReLU가 들어간다. Inveption v3의 경우 BN도 들어간다. 이것을 간단히 묶기 위해 ConvBlock 클래스 정의함.


- 각 모듈의 output 사이즈는 다음 모듈의 input 사이즈다.


- 각 인셉션 모듈 묶음 사이에는 효과적인 풀링을 하기 위한 Fig.10 인셉션 모듈(코드에서 InceptionConnect)이 들어간다. 여기서 채널 수도 다음 모듈에 맞게 늘어난다.


- 각 인셉션 모듈 안에 있는 컨볼루션 필터들엔 제로 패딩이 들어가있다.


- 각 인셉션 모듈 안에서 동작하는 컨볼루션 필터 개수는 따로 정해지지 않아서 임의로 설정한다. concat 대상인 아웃풋 필터 개수만 맞춘다.


- 각 인셉션 모듈 안에서는 인풋과 아웃풋 사이즈가 같다.


- 각 인셉션 모듈 안에서 아웃풋 쪽 제외한 그사이의 컨볼루션 통과할 때, 그 채널 수는 따로 정해지지 않아서 임의로 설정함. (ex. Fig.6의 f6 변수 같은 것)


- 1x1 컨볼루션에선 padding을 0으로 설정해야, 즉 패딩이 없어야 stride=1로 필터가 이동해도 그 크기가 유지됨. 이전 구글넷 구조 참고.


- Fig.6 & 7 인셉션 모듈에서는 (1xn), (nx1) 필터를 사용한다. 그에 따라 패딩도 직사각형 형태를 써줘야 한다. 코드 참고.


- Auxiliary classifier는 17x17 레이어 다음, 즉 Fig.6 인셉션 모듈 다음에 들어간다.


- 첫 번째 인셉션 모듈 Fig.7의 인풋은 1280이지만, 두 번째 인셉션 모듈 Fig.7 다음의 풀링 레이어에 들어가야 하는 인풋이 2048이여야 하므로, 다른 인셉션 모듈과 다르게 Fig.7은 아웃풋 사이즈가 2048이 되어야 한다.


- 논문에 padding 등의 정보가 없으면 직접 해보면서 조금씩 summary를 해보자. 특히 Fig.10 인셉션 모듈 구현할 때 그렇게 했다.

In [83]:
import torch
import torch.nn as nn


class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        x = self.conv(x)
        x = self.bn(x)
        out = self.relu(x)
        
        return out
        
        
class InceptionF5(nn.Module): # 288 = 64x3+96
    def __init__(self, in_ch): 
        super(InceptionF5, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_ch, 64, kernel_size=1, stride=1, padding=0),
            ConvBlock(64, 96, kernel_size=3, stride=1, padding=1),
            ConvBlock(96, 96, kernel_size=3, stride=1, padding=1)) # 96
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 48, kernel_size=1, stride=1, padding=0),
            ConvBlock(48, 64, kernel_size=3, stride=1, padding=1)) # 64
        
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_ch, 64, kernel_size=3, stride=1, padding=1)) # 64

        self.branch4 = nn.Sequential(
            ConvBlock(in_ch, 64, kernel_size=1, stride=1, padding=0)) # 64
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        
        return torch.cat([x1, x2, x3, x4], dim=1)
    
    
class InceptionF6(nn.Module): # 768 = 192x4
    def __init__(self, in_ch, f6): 
        super(InceptionF6, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_ch, f6, kernel_size=1, stride=1, padding=0),
            ConvBlock(f6, f6, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(f6, f6, kernel_size=(7,1), stride=1, padding=(3,0)),
            ConvBlock(f6, f6, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(f6, 192, kernel_size=(7,1), stride=1, padding=(3,0))) # 192
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, f6, kernel_size=1, stride=1, padding=0),
            ConvBlock(f6, f6, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(f6, 192, kernel_size=(7,1), stride=1, padding=(3,0))) # 192
        
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_ch, 192, kernel_size=3, stride=1, padding=1)) # 192
        
        self.branch4 = nn.Sequential(
            ConvBlock(in_ch, 192, kernel_size=1, stride=1, padding=0)) # 192
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        
        return torch.cat([x1, x2, x3, x4], dim=1)
    
    
class InceptionF7(nn.Module): # 2048 = 384x4+192+320
    def __init__(self, in_ch): 
        super(InceptionF7, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_ch, 448, kernel_size=1, stride=1, padding=0),
            ConvBlock(448, 384, kernel_size=3, stride=1, padding=1))
        
        self.branch1_1 = ConvBlock(384, 384, kernel_size=(1,3), stride=1, padding=(0,1))
        self.branch1_2 = ConvBlock(384, 384, kernel_size=(3,1), stride=1, padding=(1,0)) # 384
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 384, kernel_size=1, stride=1, padding=0))
        
        self.branch2_1 = ConvBlock(384, 384, kernel_size=(1,3), stride=1, padding=(0,1))
        self.branch2_2 = ConvBlock(384, 384, kernel_size=(3,1), stride=1, padding=(1,0)) # 384
        
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_ch, 192, kernel_size=1, stride=1, padding=0)) # 192
        
        self.branch4 = nn.Sequential(
            ConvBlock(in_ch, 320, kernel_size=1, stride=1, padding=0)) # 320
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x1 = torch.cat([self.branch1_1(x1), self.branch1_2(x1)], dim=1)
        
        x2 = self.branch2(x)
        x2 = torch.cat([self.branch2_1(x2), self.branch2_2(x2)], dim=1)
        
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        
        return torch.cat([x1, x2, x3, x4], dim=1)
    
    
class InceptionConnect(nn.Module):
    def __init__(self, in_ch, add_ch=0):           # Fig.5와 Fig.6를 연결하려면 아웃풋 768 = 240+240+288
        super(InceptionConnect, self).__init__()   # Fig.6와 Fig.7를 연결하려면 아웃풋 1280 = 256+256+768
        
         # Fig.5 & Fig.6 -> 240, Fig.6 & Fig.7 -> 256 = 240+16
        self.branch1 = nn.Sequential(
            ConvBlock(in_ch, 64+add_ch, kernel_size=1, stride=1, padding=0),
            ConvBlock(64+add_ch, 96+add_ch, kernel_size=3, stride=1, padding=1),
            ConvBlock(96+add_ch, 240+add_ch, kernel_size=3, stride=2, padding=0))
        
        # Fig.5 & Fig.6 -> 240, Fig.6 & Fig.7 -> 256 = 240+16
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 48+add_ch, kernel_size=1, stride=1, padding=0),
            ConvBlock(48+add_ch, 240+add_ch, kernel_size=3, stride=2, padding=0)) 
        
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)) # 풀링 결과(풀링 레이어에선 채널 수 안 바뀌므로 인풋 채널과 같음)
                                                              # Fig.5 & Fig.6 -> 288, Fig.6 & Fig.7 -> 768

    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        
        out = torch.cat([x1, x2, x3], dim=1)
        
        return out
    
    
class InceptionAux(nn.Module):
    def __init__(self, in_ch):
        super(InceptionAux, self).__init__()
        
        self.pool = nn.AvgPool2d(5, stride=3, padding=0)
        self.conv = nn.Conv2d(768, 128, kernel_size=1, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(3200, 1024)
        
    def forward(self, x):
        
        x = self.pool(x)
        x = self.conv(x)
        x = self.relu(x)
        x = x.view(x.size(0),-1)
        aux = self.linear(x)
        
        return aux

In [86]:
class InceptionV2(nn.Module):
    def __init__(self, num_classes = 1000):
        super(InceptionV2, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.conv4 = nn.Conv2d(64, 80, kernel_size=3, stride=1, padding=0)
        self.conv5 = nn.Conv2d(80, 192, kernel_size=3, stride=2, padding=0)
        self.conv6 = nn.Conv2d(192, 288, kernel_size=1, stride=1, padding=0)
        
        self.inceptionF5_1 = InceptionF5(288)
        self.inceptionF5_2 = InceptionF5(288)
        self.inceptionF5_3 = InceptionF5(288)
        
        self.inception_connect1 = InceptionConnect(288, add_ch=0)
        
        self.inceptionF6_1 = InceptionF6(768, f6=128)
        self.inceptionF6_2 = InceptionF6(768, f6=160)
        self.inceptionF6_3 = InceptionF6(768, f6=160)
        self.inceptionF6_4 = InceptionF6(768, f6=160)
        self.inceptionF6_5 = InceptionF6(768, f6=160)
        self.inceptionF6_6 = InceptionF6(768, f6=192)
        
        self.auxiliary = InceptionAux(768)
        
        self.inception_connect2 = InceptionConnect(768, add_ch=16)
        
        self.inceptionF7_1 = InceptionF7(1280)
        self.inceptionF7_2 = InceptionF7(2048)
        
        self.pool2 = nn.MaxPool2d(kernel_size=8, stride=1, padding=0)
        
        self.linear = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = self.pool1(x)
        
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        
        x = self.inceptionF5_1(x)
        x = self.inceptionF5_2(x)
        x = self.inceptionF5_3(x)
        
        x = self.inception_connect1(x)
        
        x = self.inceptionF6_1(x)
        x = self.inceptionF6_2(x)
        x = self.inceptionF6_3(x)
        x = self.inceptionF6_4(x)
        x = self.inceptionF6_5(x)
        x = self.inceptionF6_6(x)
        
        aux = self.auxiliary(x)
        
        x = self.inception_connect2(x)
        
        x = self.inceptionF7_1(x)
        x = self.inceptionF7_2(x)
        
        x = self.pool2(x)
        
        x = x.view(x.size(0),-1)
        
        x = self.linear(x)
        
        return aux, x

In [87]:
if __name__ == '__main__':

    from torchsummary import summary
    model = InceptionV2()
    summary(model, (3,299,299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             896
            Conv2d-2         [-1, 32, 147, 147]           9,248
            Conv2d-3         [-1, 64, 147, 147]          18,496
         MaxPool2d-4           [-1, 64, 73, 73]               0
            Conv2d-5           [-1, 80, 71, 71]          46,160
            Conv2d-6          [-1, 192, 35, 35]         138,432
            Conv2d-7          [-1, 288, 35, 35]          55,584
            Conv2d-8           [-1, 64, 35, 35]          18,432
       BatchNorm2d-9           [-1, 64, 35, 35]             128
             ReLU-10           [-1, 64, 35, 35]               0
        ConvBlock-11           [-1, 64, 35, 35]               0
           Conv2d-12           [-1, 96, 35, 35]          55,296
      BatchNorm2d-13           [-1, 96, 35, 35]             192
             ReLU-14           [-1, 96,

## 컨볼루션 계산 연습

In [88]:
# Fig.5와 Fig.6를 연결하려면 아웃풋 768 = 240+240+288
# Fig.6와 Fig.7를 연결하려면 아웃풋 1280 = 256+256+768

class InceptionF10(nn.Module):
    def __init__(self, in_ch, add_ch=0): 
        super(InceptionF10, self).__init__()
        
         # Fig.5 & Fig.6 -> 240, Fig.6 & Fig.7 -> 256 = 240+16
        self.branch1 = nn.Sequential(
            ConvBlock(in_ch, 64+add_ch, kernel_size=1, stride=1, padding=0),
            ConvBlock(64+add_ch, 96+add_ch, kernel_size=3, stride=1, padding=1),
            ConvBlock(96+add_ch, 240+add_ch, kernel_size=3, stride=2, padding=0))
        
        # Fig.5 & Fig.6 -> 240, Fig.6 & Fig.7 -> 256 = 240+16
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 48+add_ch, kernel_size=1, stride=1, padding=0),
            ConvBlock(48+add_ch, 240+add_ch, kernel_size=3, stride=2, padding=0)) 
        
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)) # 풀링 결과(인풋 사이즈와 같음): 
                                                              # Fig.5 & Fig.6 -> 288, Fig.6 & Fig.7 -> 768
        
        # 마지막 concat이 1280으로 잘 되었는지 summary로 확인하기 위함. 이걸 안하면 summary해도 최종 결과가 안보임.
        # 아웃풋은 임의로 500
        self.conv = ConvBlock(1280, 500, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        
        out = torch.cat([x1, x2, x3], dim=1)
        
        out = self.conv(out)
        
        return out

In [89]:
if __name__ == '__main__':

    from torchsummary import summary
    model = InceptionF10(768,16)
    summary(model, (768,35,35))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 80, 35, 35]          61,440
       BatchNorm2d-2           [-1, 80, 35, 35]             160
              ReLU-3           [-1, 80, 35, 35]               0
         ConvBlock-4           [-1, 80, 35, 35]               0
            Conv2d-5          [-1, 112, 35, 35]          80,640
       BatchNorm2d-6          [-1, 112, 35, 35]             224
              ReLU-7          [-1, 112, 35, 35]               0
         ConvBlock-8          [-1, 112, 35, 35]               0
            Conv2d-9          [-1, 256, 17, 17]         258,048
      BatchNorm2d-10          [-1, 256, 17, 17]             512
             ReLU-11          [-1, 256, 17, 17]               0
        ConvBlock-12          [-1, 256, 17, 17]               0
           Conv2d-13           [-1, 64, 35, 35]          49,152
      BatchNorm2d-14           [-1, 64,